# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part A | RDBMS Databases and `SQLite`

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [9]:
a = "This is a" +\
    "string"

In [11]:
a = ("This is a " +
    "string")

In [13]:
a = """
This is one line
This is another
"""

In [14]:
a

'\nThis is one line\nThis is another\n'

In [28]:
pd.io.sql.read_sql(
'''
select p.id, p.size AS size_foo_bar
    FROM properties AS p
    LIMIT 10
;
''', con = db)

,id,size_foo_bar
0,2121978635,557.0
1,89239580,1050.0
2,15131782,937.0
3,15179502,1574.0
4,52266124,1205.0
5,2100994004,400.0
6,15067755,541.0
7,15112556,1175.0
8,15133321,3094.0
9,61288341,1405.0


In [19]:
thing.head(2)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,2121978635,"829 Folsom St UNIT 906, San Francisco, CA",37781429,-122401860,0,...,557.0,sqft,NaN,None,2010.0
1,89239580,"690 Market St UNIT 1705, San Francisco, CA",37788246,-122403198,0,...,1050.0,sqft,NaN,None,2007.0


### `SELECT`

In [29]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15165953,650000.00,$
1,80749447,1.15,$M
2,15155751,665000.00,$
3,15143887,2.10,$M
4,15117639,1.35,$M
...,...,...,...
995,69819708,731000.00,$
996,15076156,5.53,$M
997,119685619,625000.00,$
998,15113584,895000.00,$


### `WHERE`

In [36]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000 / 1000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
420,119685503,0.750
421,69819708,0.731
422,119685619,0.625
423,15113584,0.895


> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [40]:
df_1= pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

df_6= pd.io.sql.read_sql(
'''
SELECT id, sale_price  AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

df = pd.concat([df_1, df_6])

### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [42]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price_foo
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, id AS foo_bar, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



DatabaseError: Execution failed on sql '
SELECT id, sale_price / 1000000 AS sale_price_foo
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, id AS foo_bar, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
': SELECTs to the left and right of UNION ALL do not have the same number of result columns

### `WITH`

In [ ]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (....)

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

### `JOIN`

In [43]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `WITH` (take 2)

In [45]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [48]:
pd.io.sql.read_sql(
'''
SELECT COUNT() AS count
    FROM properties
    GROUP BY beds
;
''', con = db)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,80738317,"650 Delancey St APT 109, San Francisco, CA",37782577,-122389991,0,...,1554.0,sqft,NaN,None,1996.0
1,82786211,"310 Townsend St APT 311, San Francisco, CA",37777027,-122395736,0,...,853.0,sqft,NaN,None,2006.0
2,54854296,"720 Stockton St APT 3, San Francisco, CA",37792578,-122407366,0,...,886.0,sqft,NaN,None,2001.0
3,15195183,"3916 Alemany Blvd, San Francisco, CA",37711527,-122467755,0,...,1300.0,sqft,2553.0,sqft,1941.0
4,15180783,"430 Fair Oaks St, San Francisco, CA",37749725,-122424094,0,...,2678.0,sqft,NaN,None,1911.0
5,15181250,"4069 25th St, San Francisco, CA",37749628,-122430985,0,...,1837.0,sqft,2848.0,sqft,1905.0
6,15076156,"1533 Sutter St, San Francisco, CA",37786658,-122426481,0,...,7375.0,sqft,2748.0,sqft,1890.0
7,15072504,"2660 Scott St, San Francisco, CA",37794473,-122439703,0,...,9190.0,sqft,5623.0,sqft,1908.0
8,15095541,"3321 Anza St, San Francisco, CA",37777956,-122483984,0,...,4729.0,sqft,2430.0,sqft,1905.0
9,2099976986,"639 35th Ave, San Francisco, CA",37776897,-122495582,0,...,2790.0,sqft,3005.0,sqft,1900.0


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [50]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT baths, AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

,baths,mean_sale_price
0,NaN,1.739026
1,1.00,0.987656
2,1.10,1.420000
3,1.25,1.600000
4,1.50,1.223378
...,...,...
16,6.50,16.000000
17,7.00,0.999000
18,7.50,5.530000
19,8.00,13.100000


Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [60]:
pd.io.sql.read_sql(
'''
WITH t AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT
        AVG(t.sale_price)
    FROM
        properties p        
       JOIN  t on p.id = t.id
    WHERE p.is_a_studio = 1
;
''', con = db)

,AVG(t.sale_price)
0,1.464034


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [71]:
pd.io.sql.read_sql(
'''
SELECT a.id AS id_a, b.id AS is_b, a.address AS address_a, b.address AS address_b
    FROM properties AS a
    JOIN properties AS b ON b.latitude = a.latitude AND b.longitude = a.longitude
    WHERE a.id > b.id 
;
''', con = db)

,id_a,is_b,address_a,address_b
0,119685526,119685188,"1902 Filbert St, San Francisco, CA","1902A Filbert St, San Francisco, CA"
1,15076711,15076699,"1 Daniel Burnham Ct APT 1404, San Francisco, CA","1 Daniel Burnham Ct APT 1202, San Francisco, CA"
2,79846666,79844263,"555 4th St UNIT 513, San Francisco, CA","555 4th St, San Francisco, CA"
3,80730869,69819664,"260 King St UNIT 817, San Francisco, CA","260 King St UNIT 1205, San Francisco, CA"
4,80730869,69819845,"260 King St UNIT 817, San Francisco, CA","260 King St UNIT 769, San Francisco, CA"
...,...,...,...,...
103,111931130,111930969,"860 Waller St APT 1, San Francisco, CA","860 Waller St APT 2, San Francisco, CA"
104,2100850402,125156575,"1915 Diamond St, San Francisco, CA","1915 Diamond St, San Francisco, CA"
105,61288454,60825591,"77 Dow Pl APT 507, San Francisco, CA","77 Dow Pl APT 500, San Francisco, CA"
106,67395015,63107226,"8400 Oceanview Ter APT 218, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
